In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from scipy.stats import norm

KeyboardInterrupt: 

## Load results of MC simulation

In [ ]:
with open('results_dict.pkl', 'rb') as pickle_file:
    results_dict = pickle.load(pickle_file)

## Assess convergence rates and asymptotic properties

In [ ]:
theta_0 = 0.5   # true ATE
sample_sizes = list(results_dict.keys())
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']
names = ['Infeasible', 'DML', 'Regression', 'IPW']

In [ ]:
# Visual comparison of estimators
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(12, 6.5))

for i, (N, results) in enumerate(results_dict.items()):
    ax = axes.flat[i]
    ax.scatter(theta_0, 0, marker=10, color='dimgrey', label='ATE')
    ate_estimates = results[0]
    for k in [0, 2, 3, 1]:
        sns.kdeplot(ate_estimates[:,k], ax=ax, color=colors[k], label=names[k])
    ax.set_title(r'$N=$' + f'{N}')
    
    if not (i==0 or i==4):
        ax.set_ylabel('')
    #if i==0:
    #    ax.set_xlim(left=0.33, right=1.77)
    #elif i==1:
    #    ax.set_xlim(left=0.535, right=1.545)
    #elif i==2:
    #    ax.set_xlim(left=0.7, right=1.385)
    #elif i==3:
    #    ax.set_xlim(left=0.775, right=1.27)
    #elif i==4:
    #    ax.set_xlim(left=0.845, right=1.185)
    #elif i==5:
    #    ax.set_yticks([0, 5, 10, 15, 20])
    ax.set_xlim(left=-0.5, right=1.5)
    if i==6: 
        handles, labels = ax.get_legend_handles_labels()

order = [0, 1, 4, 2, 3] 
axes.flat[-1].legend([handles[j] for j in order], [labels[j] for j in order], 
                     loc='center left', bbox_to_anchor=(0, 0.5))        
axes.flat[-1].axis('off')
fig.tight_layout()

for fmt in ['png', 'pdf']:
    fig.savefig(f'figures/Visual_comparison.{fmt}', dpi=300, bbox_inches='tight')

In [ ]:
# Convergence rates
rmse = np.empty((len(sample_sizes), 4))

for i, (N, results) in enumerate(results_dict.items()):
    ate_estimates = results[0]
    rmse[i] = np.sqrt(np.mean((ate_estimates-theta_0)**2, axis=0))

plt.figure(figsize=(6, 4.5))
for k in range(4):
    plt.plot(np.log(sample_sizes), np.log(rmse[:,k]), color=colors[k], label=names[k])
plt.xlabel(r'$\log N$')
plt.ylabel(r'$\log$' + 'RMSE')
plt.legend()

for fmt in ['png', 'pdf']:
    plt.savefig(f'figures/Convergence_rates_1.{fmt}', dpi=300, bbox_inches='tight')

In [ ]:
# Convergence rates
slopes = np.polyfit(np.log(sample_sizes), np.log(rmse), 1)[0,:]

plt.figure(figsize=(6, 4.5))
plt.bar(names, -slopes, color=colors)
plt.axhline(y=0.5, color='dimgrey', linestyle='--')
plt.ylabel('RMSE convergence rate')

for fmt in ['png', 'pdf']:
    plt.savefig(f'figures/Convergence_rates_2.{fmt}', dpi=300, bbox_inches='tight')

In [ ]:
-slopes

In [ ]:
# Asymptotic normality
grid = np.linspace(-4, 4, 100)
pdf_values = norm.pdf(grid)

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(12, 6.5))

for i, (N, results) in enumerate(results_dict.items()):
    ax = axes.flat[i]
    ate_estimates, sigma_estimates, _, _ = results
    ax.plot(grid, pdf_values, color='dimgrey', label=r'$\mathcal{N}(0,1)$')
    sns.kdeplot(np.sqrt(N)*(ate_estimates[:,1]-theta_0)/sigma_estimates, ax=ax, color=colors[1],
                label=r'DML: $\frac{\sqrt{N}(\hat{\theta}_0-\theta_0)}{\hat{\sigma}}$')
    ax.set_xlim(left=-4.75, right=4.75)
    ax.set_ylim(bottom=None, top=0.423)
    ax.set_title(r'$N=$' + f'{N}')
    
    if not (i==0 or i==4):
        ax.set_ylabel('')
    if i==0:
        ax.set_xticks([-4, -2, 0, 2, 4])    
    if i==6: 
        handles, labels = ax.get_legend_handles_labels()
  
axes.flat[-1].legend(handles, labels, loc='center left', bbox_to_anchor=(0, 0.5))        
axes.flat[-1].axis('off')
fig.tight_layout()

for fmt in ['png', 'pdf']:
    fig.savefig(f'figures/Asymptotic_normality.{fmt}', dpi=300, bbox_inches='tight')

In [ ]:
# Coverage probability
cov_prob_list = []

for results in results_dict.values():
    CIs = results[2]
    cov_prob = np.mean((theta_0>=CIs[:,0]) & (theta_0<=CIs[:,1]))
    cov_prob_list.append(cov_prob)

plt.figure(figsize=(6, 3.5))
plt.axhline(y=0.95, color='dimgrey', linestyle='--', label='Nominal')
plt.plot(sample_sizes, cov_prob_list, color=colors[1], label=names[1])

plt.xlabel(r'$N$')
plt.ylabel('Coverage probability')
#plt.yticks([0.930, 0.935, 0.940, 0.945, 0.950])
#plt.ylim(bottom=0.929, top=0.951)
plt.legend(loc='lower right')

for fmt in ['png', 'pdf']:
    plt.savefig(f'figures/Coverage_probability.{fmt}', dpi=300, bbox_inches='tight')